In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("proj3.ipynb")

# Project 3: Data Transformation

## Due Date: Tuesday 04/13, 11:59 PM

## Assignment Details

In this project, we'll be working with one month of data from sensors in buildings at UC Berkeley. This is a very typical real-world dataset---i.e. it's kind of a mess. The full dataset contains a giant `data` table of sensor readings that is many billions of readings over the course of a decade; we will look at a single month of that data. It also contains a variety of other tables that contextualize the readings.

The schema for the database is shown below. Sometimes people think that if the data is a nice schema, then it's ready to go! We'll see about that.

<img src="files/schema.png">

## Scoring Breakdown
Question | Points
--- | ---
1a	| 1
1b  | 1
1c	| 1
1d	| 1
1e	| 1
2a	| 3
2b	| 1
3a	| 0
3b	| 1
3c	| 1
3d  | 0
3e  | 2
4a	| 2
4b	| 2
4c	| 3
5a | 1
5b | 1
5c | 3
**Total** | 25

In [2]:
# Run this cell to set up imports
import numpy as np
import pandas as pd

In [3]:
%reload_ext sql
%sql postgresql://jovyan@127.0.0.1:5432/template1

## Loading Up the Database
To load the database, run the following cell.

In [4]:
import subprocess
import os
import warnings

call = subprocess.run(["psql", "-h", "localhost", \
                       "-tAc", "SELECT 1 FROM pg_database WHERE datname='ucb_buildings'", "template1"], \
                      stdout=subprocess.PIPE, text=True)

if call.stdout != "1\n":
    os.system("gunzip -c proj3.sql.gz | psql -h localhost -d template1 -f -")
else:
    warnings.warn("you need to run dropdb -h localhost ucb_buildings if you want to reload the database.")
%sql postgresql://jovyan@127.0.0.1:5432/ucb_buildings

Run the following cell for grading purposes.

In [4]:
!mkdir -p results

<!-- BEGIN QUESTION -->

## Question 1: Unboxing the Data

### Question 1a

Note that the `data` table, in the full database, is billions of rows. What do you notice about the design of the database schema that helps support the large amount of data?

<!--
BEGIN QUESTION
name: q1a
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 1b

Because this data is already in a database, we have a bunch of useful high-level schematic information. But we're still missing some info we might want.

Assuming no new inserts, is there a potentially useful key in the `buildings_site_mapping` table? That is, is there a subset of columns that---at least for the provided data---is a unique ID for each row? Write a SQL query that returns a single row with one column of boolean value `true` if there is a unique ID per row, or a single row with one column of value `false` otherwise. The output column name in your query can be anything.

<!--
BEGIN QUESTION
name: q1bm
manual: true
points: 0
-->

In [6]:
%%sql result_1b <<
...

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q1b
points: 1
-->

In [14]:
# Do not delete/edit this cell
result_1b.DataFrame().to_csv('results/result_1b.csv', index=False)

In [ ]:
grader.check("q1b")

### Question 1c

The diagram claims that `buildings_site_mapping` has a many-to-many relationship with `real_estate_metadata`. Let's validate that. 

Below is an example of `json_agg` being used with a table; you will need to do this in the next two parts.

In [21]:
%%sql
SELECT b.site, json_agg(b) from buildings_site_mapping b GROUP BY b.site LIMIT 5;

Find the values of `buildings_site_mapping.building` that match multiple tuples in `real_estate_metadata.building_name`, and for each such value of `buildings_site_mapping.building` return the matches as JSON via `json_agg(real_estate_metadata)`. Your output should contain the building and the `json_agg` in that order. Order your final result by building.

In [30]:
%%sql result_1c <<
...

In [32]:
# Do not delete/edit this cell
result_1c.DataFrame().to_csv('results/result_1c.csv', index=False)

In [ ]:
grader.check("q1c")

### Question 1d

Now find examples of many matches in the opposite direction. For each `real_estate_metadata.building_name` value, find the ones that have multiple matches in `buildings_site_mapping.building`, and for each return a `json_agg` of the multiple values for `buildings_site_mapping`. Your output should contain the building name and the `json_agg` in that order. Order your final result by building name.

In [43]:
%%sql result_1d <<
...

In [44]:
# Do not delete/edit this cell
result_1d.DataFrame().to_csv('results/result_1d.csv', index=False)

In [ ]:
grader.check("q1d")

<!-- BEGIN QUESTION -->

### Question 1e

Looking at the output of the previous question, what do you notice about the entries of the `json_agg` column? Are there any duplicates within the entries?

<!--
BEGIN QUESTION
name: q1e
manual: true
points: 1
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



## Question 2: Looking for Outliers in the Readings
Physical sensors like the ones generating this data are notorious for producing crazy outliers on occasion. In this section we'll do a little data cleaning of the outliers.

All the readings from all different kinds of sensors are mixed together in the `data` table. This hodgepodge of mixed readings is going to require us to do some extra work to look for outliers, which we didn't quite see in class. Let's get started.

### Question 2a: Outlier Detection

Let's find the outlying values *for each sensor id*. We'll call something an outlier if it is **3 Hampel X84 intervals** away from the median.

Specifically, create a view `labeled_data` that joins up all columns in `data` and `metadata`, and adds three additional columns at the far right:
  - `median` containing the median using `percentile_disc`
  - `mad` containing the mad,
  - `outlier` that contains `true` for the outlier readings and `false` for the rest. **Also,** for data points where the mad is 0, set this to `false`.

In [88]:
%%sql result_2a <<
CREATE OR REPLACE VIEW labeled_data AS
...
;
SELECT * FROM labeled_data WHERE outlier ORDER BY id, time LIMIT 100;

In [89]:
# Do not delete/edit this cell
result_2a.DataFrame().to_csv('results/result_2a.csv', index=False)

In [ ]:
grader.check("q2a")

### Question 2b: Outlier Handling (Winsorization)

In this step we'll define a view `cleaned_data` over all the columns of `labeled_data` and one additional column on the far right called `clean_value`. This column will contain a copy of `data.value` if that value is not an outlier. For outliers, it should contain the value Winsorized to the nearest outlier boundary value (3 Hampel X84 intervals from the median). If the MAD is 0, then the cleaned value should be the same as the original value.

In [96]:
%%sql result_2b <<
CREATE OR REPLACE VIEW cleaned_data AS
...
;
SELECT * FROM cleaned_data WHERE outlier ORDER BY id, time LIMIT 100;

In [97]:
# Do not delete/edit this cell
result_2b.DataFrame().to_csv('results/result_2b.csv', index=False)

In [ ]:
grader.check("q2b")

<!-- BEGIN QUESTION -->

## Question 3: Entity Resolution

### Question 3a
There is a lot of mess in this dataset related to entity names. As a start, have a look at all of the distinct values in the `units` field of the `metadata` table. What do you notice about these values? Are there any duplicates?

<!--
BEGIN QUESTION
name: q3a
manual: true
points: 0
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3b

Sometimes, entity resolution is as simple as a text transformation. For example, how many unique `units` values are there, and how many would there be if we ignored case (upper vs. lower case)? Your output should be a table with one row and two columns; the first column should contain the number of unique `units` values, and the second column should contain the number of unique `units` values if we ignored case.

<!--
BEGIN QUESTION
name: q3bm
manual: true
points: 0
-->

In [32]:
%%sql result_3b <<
...

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3b
points: 1
-->

In [40]:
# Do not delete/edit this cell
result_3b.DataFrame().to_csv('results/result_3b.csv', index=False)

In [ ]:
grader.check("q3b")

<!-- BEGIN QUESTION -->

### Question 3c

Arguably we shouldn't care about these alternative unit labels, *as long as each sensor class uses a single value of `units` for all its sensor ids*. After all, maybe the capitalization means something to somebody!

Write a SQL query that returns single row with one column of value `true` if the condition in italics above holds, or a single row with one column of value `false` otherwise.

<!--
BEGIN QUESTION
name: q3cm
manual: true
points: 0
-->

In [22]:
%%sql result_3c <<
...

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q3c
points: 1
-->

In [29]:
# Do not delete/edit this cell
result_3c.DataFrame().to_csv('results/result_3c.csv', index=False)

In [ ]:
grader.check("q3c")

<!-- BEGIN QUESTION -->

### Question 3d

Moving on, have a look at the `real_estate_metadata` table---starting with the distinct values in the `location` field! What do you notice about these values?

<!--
BEGIN QUESTION
name: q3d
manual: true
points: 0
-->

_Type your answer here, replacing this text._

<!-- END QUESTION -->



### Question 3e

It turns out this table was the result of an [OCR scan](https://en.wikipedia.org/wiki/Optical_character_recognition). We'll just clean up the `location` column for now, and leave you to imagine the effort to do a full cleanup of all columns.

To provide some useful utility functions, we have preloaded Postgres' extension packages for "fuzzy" string matching and trigrams for you. You can use any of the string functions in those packages if you like ([as documented here for fuzzystrmatch](https://www.postgresql.org/docs/current/fuzzystrmatch.html) or [here for pg_trgm](https://www.postgresql.org/docs/current/pgtrgm.html)).

We also created a lookup table of canonical names, `uc_locations`.

Now, using any of the string functions you like (or none at all!), create a view that has one extra column `clean_location`. That column should contain the best match from `uc_locations.loc_name`. You may find that you can't clean up everything with the string functions, so your view may have to include some specific logic for cases in the data that have to be handled "manually". You can choose to do this question in whatever manner you wish as long as your query does not use `CREATE TABLE`, `INSERT INTO`, or `UPDATE`.

In [212]:
%%sql result_3e <<
...

In [237]:
# Do not delete/edit this cell
result_3e.DataFrame().sort_values(['clean_location', 'building', 'building_name']).iloc[::10].to_csv('results/result_3e.csv', index=False)

In [ ]:
grader.check("q3e")

In [8]:
%%sql
SELECT * from gaps ORDER by id, time LIMIT 20;

## Question 4: Interpolating Missing Data
Real-world data, real-world problems. Our sensors should be reporting every 15 minutes, but you can be sure that we're missing some data. Here we will fix it. It's a bit more involved than what we looked at in class!

### Question 4a: Finding missing readings
In the `data` table, the `id` column identifies a unique sensor. Sensor readings should be recorded every 15 minutes from every sensor. Are we missing any readings, and if so which ones? We will focus on readings that are separated by at least 30 minutes or more; readings that are \[0-30) minutes apart are considered to be fine.

To answer this question you'll need to read up a bit on [SQL timestamps](https://www.postgresql.org/docs/current/datatype-datetime.html) and [Functions for manipulating datetime types](https://www.postgresql.org/docs/current/functions-datetime.html). Have a particular look at the following:
- The [date_trunc](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC) function will quantize times to the nearest unit of your choosing. E.g. to round the `time` field to the nearest minute you can say `date_trunc('minute', time)`. **You'll need to quantize to minutes right away before you worry about missing readings.**
- There are various ways to enter constant intervals of time as strings. E.g. a 30 minute interval can be written as `interval '30 minutes'` or `'30 minutes'::interval`. See [date/time input](https://www.postgresql.org/docs/current/datatype-datetime.html#DATATYPE-INTERVAL-INPUT) for more info.
- You can do arithmetic on date/time types [as documented here](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-TRUNC). That will handle all the weird periodicities of clocks and calendars for you. Pay attention to the input and output types of these functions!
- Alternatively, the [EXTRACT](https://www.postgresql.org/docs/current/functions-datetime.html#FUNCTIONS-DATETIME-EXTRACT) function is sometimes handy. Note the special `EXTRACT(EPOCH FROM ...)` case. This converts a timestamp into an integer representing the number of seconds since midnight, 1/1/1970 (the dawn of [UNIX time](https://en.wikipedia.org/wiki/Unix_time)!)  You can do normal integer comparisons and arithmetic on the results.


Create a view called `gaps` that returns pairs of `data` tuples per sensor id that are separated by 30 minutes or more. The output should augment the `data` schema with three columns:
- `lagtime` is the quantized time of the previous reading for that sensor (relative to the current row for a particular row)
- `lagvalue` is the value of the previous reading for that sensor
- `timediff` is the difference in quantized time between this reading and the previous reading

In [6]:
%%sql result_4a <<
CREATE OR REPLACE VIEW gaps AS
...
;
SELECT * FROM gaps ORDER BY id, time LIMIT 10;

In [261]:
# Do not delete/edit this cell
result_4a.DataFrame().to_csv('results/result_4a.csv', index=False)

In [ ]:
grader.check("q4a")

### Question 4b: Creating tuples for the missing readings
Now we need to manufacture new tuples to fill in the gaps. For example, if you had a tuple from id `abc` timestamped at 1PM today and the next tuple in time from `abc` was timestamped at 1:45PM, you'll need to manufacture two new tuples with id `abc` and `NULL` values: one timestamped at 1:15PM and another timestamped at 1:30PM. We will worry about replacing the `NULL` values in the next step.

To manufacture tuples not related to stored data in the database, we'll need to use a *table-valued function* as we did in lecture 12 (when we manufactured data from a normal distribution). The table-valued function we want here is `generate_series` [(documented here)](https://www.postgresql.org/docs/current/functions-srf.html), which we will use to generate *and sequentially timestamp* the right number of tuples to match the number of tuples we found missing.

To get a feel for `generate_series`, consider the following simple query that generates a table of integers with intervals of size 3 between them.

In [ ]:
%%sql
SELECT *
  FROM generate_series(1, 10, 3);

Now, we can use `generate_series` in a `LATERAL JOIN` query: for each tuple on the left of the `LATERAL` it will produce a series based on the values of that tuple. So for example, we can generate 2 tuples for each tuple of `uc_locations` as follows:

In [ ]:
%%sql
SELECT loc_id, loc_name, length(loc_name), newval
  FROM uc_locations u, 
       LATERAL generate_series(length(loc_name), length(loc_name) + 2, 2) AS newval;

Notice how the 2 values it generates are the length of the `loc_name`, and the length + 2. You might want to play with the query above to make sure you understand the documentation for `generate_series` and `LATERAL`.

Ok, on to your task!

Create a view `complete` that contains the tuples from `data` as well as new tuples that fill in any gaps greater than 30 minutes. Each gap should be filled by adding tuples in increments of 15 minutes from the *start* of the gap, with `NULL` as the value. You probably want to use your `gaps` view as well as `generate_series` to do this!

In [46]:
%%sql result_4b <<
CREATE OR REPLACE VIEW complete AS
...
;

SELECT * FROM complete ORDER BY id, time LIMIT 100;

In [285]:
# Do not delete/edit this cell
result_4b.DataFrame().to_csv('results/result_4b.csv', index=False)

In [ ]:
grader.check("q4b")

### Question 4c: Linear Interpolation
*Note: If you struggled with Steps 1 and 2 of this problem, you can use our table `complete_provided` instead of your `complete` table in Step 3.*

Now, given the `complete` view or the `complete_provided` table, the remaining task is to do linear interpolation to fill in the missing values we manufactured in Step 2. We have code from Lecture 13 we can reuse here! In particular, your database already includes the UDA `coalesce_agg` we used in lecture.

But note that in Lecture 13's example of linear interpolation we had a field called `feature_id` that had two convenient properties:
1. `feature_id` was used to order *all* the records in the table. By contrast, here the ordering we care about is the series of timestamps for each sensor `id` *independently*.
2. `feature_id` was a gap-free sequence of incrementing integers. Coupled with the previous point, that allowed us to use arithmetic on `feature_id` to calculate the distance between records in order in the "backward" pass. We don't have any such field handy here, so you'll have to find some other way (hint: window function!) to achieve the same effect where you need it.

These two points will require you to adapt the linear interpolation code from class to work here.

Create a view `likely_data` that contains all the tuples from `complete`, with an additional column called `interpolated` that contains a copy of `value` if it is non-NULL, otherwise an interpolated value based on linear interpolation *per sensor id over time*. The three cells below correspond to the forward, backward, and final passes from lecture.

In [76]:
%%sql
CREATE OR REPLACE VIEW forward AS
...

In [77]:
%%sql
CREATE OR REPLACE VIEW backward AS
...

In [78]:
%%sql result_4c <<
CREATE OR REPLACE VIEW likely_data AS
...
;
SELECT * FROM likely_data WHERE run_size > 2 ORDER BY id, time LIMIT 100;

In [79]:
# Do not delete/edit this cell
result_4c.DataFrame().to_csv('results/result_4c.csv', index=False)

In [ ]:
grader.check("q4c")

## Question 5: Granularity Transforms
In this question we will write a roll-up query on an ontology. This requires a bit of background explanation.

### The Brick Ontology
The ongoing [Software-Defined Buildings](http://sdb.cs.berkeley.edu/sdb/) research project at Berkeley has led the development of a standard ontology for building metadata called [Brick](https://docs.brickschema.org/intro.html) that is getting a fair bit of attention in the world of IoT. Like many ontologies, it is represented as triples `(subject, predicate, object)`. In our database the Brick ontology has been stored in a table called `ontology`.

### The `SubClassOf` Predicate and the `Sensor` Class
We are interested in readings from different classes of sensor devices. More specifically, we are interested in rows from the `metadata` table whose `metadata.class` entry maps to an `ontology` subject $s$, and that subject is in an ontology tuple `(`$s$`, http://www.w3.org/2000/01/rdf-schema#subClassOf, https://brickschema.org/schema/Brick#Sensor)`. Then we know that the sensor in `metadata` belongs to a sub-class of `Sensor`. 

The diagram below shows a few of the `subject`s and `object`s from `ontology` in ovals. There is a dark arrow between two ovals if there is a corresponding row in `ontology`. The sub-classes of `Sensor` in the diagram are shown in yellow; we'll call them "Sensor children".

### The `transitive_subClassOf` Relation
Intuitively, the subclasses of the "Sensor children" are also themselves sensors; and the subclasses of those classes are also sensors, and so on. That is, we're really interested in the [transitive closure](https://en.wikipedia.org/wiki/Transitive_closure) of the `subClassOf` predicate. We can form transitive "chains" like this by joining `ontology` with itself. For example, the query 
```sql
SELECT o1.subject, o2.object 
  FROM ontology o1 INNER JOIN ontology o2 ON o1.object = o2.subject
 WHERE o1.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'
   AND o2.predicate = 'http://www.w3.org/2000/01/rdf-schema#subClassOf'
``` 
forms edges between the endpoints of chains of length 2.

Extending this example, computing edges of length 3 requires joining 3 references to `ontology`, and so on. To form all chains of *arbitrary* length requires the use of a *recursive query*---something we haven't learned in this class. So we have provided you a materialized view called `transitive_subClassOf` that provides the result of that recursive query. It contains tuples of the form `(object, subject, hops, path)` where `subject` and `object` are connected transitively in `ontology` via one or more `subClassOf` predicates described above. `path` is a Postgres array type that shows the transitive path of class names through the ontology from `subject` to `object`, and `hops` is the length of that path. In the figure below, there is an aquamarine edge from one node to another if there is a row in `transitive_subClassOf` for that pair. For example, there is a row:

| object | subject | hops | path |
| :-- | :-- | :-- | :-- |
| `https://brickschema.org/schema/Brick#Sensor` | `https://brickschema.org/schema/Brick#CO2_Level_Sensor` | `3`  | `{https://brickschema.org/schema/Brick#CO2_Level_Sensor,`
| | | | `https://brickschema.org/schema/Brick#CO2_Sensor,`
| | | | `https://brickschema.org/schema/Brick#Particulate_Matter_Sensor,`
| | | | `https://brickschema.org/schema/Brick#Sensor}`

<img src="files/subClass.png">

*Just for fun: If you're curious about the recursive query that computes this view, you can issue the command `\d+ transitive_subClassOf` to Postgres. You may also want to read the documentation for SQL's [WITH RECURSIVE](https://www.postgresql.org/docs/9.1/queries-with.html) clause as implemented in Postgres.*

<!-- BEGIN QUESTION -->

### Question 5a

We want to check the graph properties of the `subClassOf` predicate. It would be confusing if the `subClassOf` predicate had cycles!

Write a query on `transitive_subClassOf` to check for cycles. Ask yourself this: what property in `transitive_subClassOf` would be a "witness" to a cycle?? Your query should return one row of one boolean column: `true` if the predicate has cycles, `false` otherwise.

<!--
BEGIN QUESTION
name: q5am
manual: true
points: 0
-->

In [93]:
%%sql result_5a <<
...

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q5a
points: 1
-->

In [94]:
# Do not delete/edit this cell
result_5a.DataFrame().to_csv('results/result_5a.csv', index=False)

In [ ]:
grader.check("q5a")

<!-- BEGIN QUESTION -->

### Question 5b

Assuming it's not cyclic, the next question is whether the `subClassOf` predicate forms *tree-shaped* connections only. The signature of a tree is that each node has at most one outbound edge (pointing to its "parent"). If any node has multiple outbound edges, the predicate forms a more general directed acyclic graph (a DAG). So we are looking to see if each subject is in a `subClassOf` predicate *with at most one object* (the single parent in the tree). 

Write a query that returns `true` if each subject is in a `subClassOf` predicate with at most one object, and `false` otherwise.

<!--
BEGIN QUESTION
name: q5bm
manual: true
points: 0
-->

In [98]:
%%sql result_5b <<
...

<!-- END QUESTION -->

<!--
BEGIN QUESTION
name: q5b
points: 1
-->

In [99]:
# Do not delete/edit this cell
result_5b.DataFrame().to_csv('results/result_5b.csv', index=False)

In [ ]:
grader.check("q5b")

### Question 5c

Now that we understand the graph properties of the ontology, let's use the ontology to do a roll-up as intended. We're interested in the number of unique sensor `id`s from `metadata` that are transitively in subclasses of each "Sensor child" class. To compute this, you will have to associate each `metadata.id` with a matching `brickclass` *(if there is one!)*, and use the `transitive_subClassOf` view to identify all "Sensor children" for which the matching `brickclass` is transitively in a `subClass`. (If the `subClass` predicate is a DAG, then each `id` should be counted for *all* the direct subclasses of `Sensor` that it's transitively underneath.)

Write a query that returns tuples of the form `(sensor_child, count)` that returns for each "Sensor child" the count of *distinct* `metadata.id` entries that are subclasses of that "Sensor child" class. Only output tuples that have a matching `brickclass`.

In [109]:
%%sql result_5c <<
...

In [110]:
# Do not delete/edit this cell
result_5c.DataFrame().to_csv('results/result_5c.csv', index=False)

In [ ]:
grader.check("q5c")

## Congratulations! You have finished Project 3.

Run the following cell to zip the results of your queries. You will also need to run the export cell at the end of the notebook. **For submission on Gradescope, you will need to submit both the proj3.zip file generated by the export cell and the results.zip file generated by the following cell.**

In [ ]:
!zip -r results.zip results

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()